In [1]:
import numpy as np
import sys
import toyplot
import toyplot.browser

sys.path.append("..")
from commsim.Species import Species
from commsim.Environment import Environment

In [2]:
class Species:
    """
    A species origin is the x,y coordinate defining the center of
    its geographic range. The environment at this location will 
    determine its niche preferences, which combined with a dispersal
    decay rate, and other species occurrences, will determine its 
    occurrence in sites around origin.
    
    Parameters
    ----------
    origin (tuple or list):
        A set of x, y coordinates for the center of origin of a spp. range.
    decay (float):
        The exponential decay rate parameter for dispersal from origin.
    bioclim (dict):
        A dict of key,val pairs with keys as bioclim variable names and 
        values as tuples of (mean, std) for species tolerances.
    """
    # global species index counter
    idx = 0
    
    def __init__(self, origin, decay, bioclim):
       
        # store this instances attrs
        self.idx = Species.idx
        self.origin = origin
        self.decay = decay
        self.bioclim = bioclim
        
        # increment the global counter when an instance is made
        Species.idx += 1        
    
    def __repr__(self):
        """
        custom repr shows species global idx, origin, and decay"
        """
        return f"<Species idx={self.idx}, o={self.origin}, e={self.decay}, bioclim={self.bioclim}>"




if __name__ == "__main__":

    # two example Species 
    A = Species((10, 10), 0.5, {"mean-temp": (5, 1)})
    B = Species((30, 30), 0.05, {"mean-temp": (3, 3)})
    print(A)
    print(B)

<Species idx=0, o=(10, 10), e=0.5, bioclim={'mean-temp': (5, 1)}>
<Species idx=1, o=(30, 30), e=0.05, bioclim={'mean-temp': (3, 3)}>


In [3]:
ped_siphonantha = Species((42, 42), 0.3, {"mean-temp": (8, 2)})

In [4]:
print(ped_siphonantha)

<Species idx=2, o=(42, 42), e=0.3, bioclim={'mean-temp': (8, 2)}>


In [5]:
class Environment:
    """
    A matrix of environment layers with float values representing 
    spatial variation. Environmental variables (bioclims) are added
    as simple gradients with min,max values, and they are then 
    transformed by elevation which can apply a different coefficient
    to each variable (how strongly it correlates with elevation).

    The environmental matrix is created during init and then there
    are draw functions available to visualize the array layers.
    
    Parameters:
    -----------
    ...
    """
    def __init__(self, shape, bioclims, elevation):
              
        # env layers; starts with 1, additional can be added.
        nbioclim = max(1, len(bioclims))
        self.arr = np.zeros((nbioclim, shape[0], shape[1]), dtype=float)
        
        # elevation multipliers
        self.elevation = elevation
        self.elev = np.zeros((1, shape[0], shape[1]), dtype=float)        

        # fill base gradient layers first
        self._apply_base_gradient_layers()

        # then transform base layers by elevation
        self._apply_elevation_transform()

    def __repr__(self):
        return f"<Environment shape=({self.arr.shape})>"


    def _apply_base_gradient_layers(self):
        pass

    def _apply_elevation_transform(self):
        pass
        
    
    def add_linear_gradient(self):
        """
        Adds a linear gradient niche axis (e.g., temperature from S to N)
        """
        pass
    
    
    def add_gaussian_gradient(self, npeaks, seed):
        """
        Adds N gaussian gradients summed (e.g., mountain elevational variation)
        """
        pass
        
        
    def add_chevron_layers(self, nshapes):
        """
        Adds N chevrons to fit left to right on grid.
        """
        pass
    
    
    def get_summed_layers(self):
        """
        Return summed and normalized environmental layers.
        """
        pass
    
    
    def draw(self):
        """
        Returns a toyplot (canvas, axes) tuple of grid env.
        """

        # build drawing
        canvas, table = toyplot.matrix(self.arr[0])

        # show result in browser if not in a notebook
        if not sys.stdout.isatty():
            toyplot.browser.show(canvas)
            return (None, None)
        return canvas, table
            

if __name__ == "__main__":

    env = Environment(
        shape=(100, 100), 
        bioclims={
            "mean-temp": (0, 20, "linear-sn"),
            "min-temp": (-20, 10, "linear-we"),
            "mean-precip": (0.1, 1.0, "linear-we"),
        },
        elevation={
            "mean-temp": 0.1,
            "min-temp": 0.2,
            "mean-precip": 0.5,
        },
    )

    print(env)
    env.draw()

<Environment shape=((3, 100, 100))>


In [6]:
"""
Community class object for representing multiple species ranges.
"""

class Community:
    """
    Generates a community assemblage matrix from a commsim.Environment
    object and an list of commsim.Species objects.
    """

    def __init__(self, env, species_list):

        # get dimensions from the environment matrix
        self.env = env
        self.x = env.arr.shape[1]
        self.y = env.arr.shape[2]

        # make new array for storing all species ranges
        self.arr = np.zeros((len(species_list), self.x, self.y), dtype=np.int8)

    def __repr__(self):
        return f"<Community nspecies={self.arr.shape[0]}; nenvs={self.env.arr.shape[0]}>"

    def spp_richness(self):
        "returns the sum of species at all sites"
        pass

    def spp_range(self, idx):
        "returns the occurrence of a selected species"
        pass


if __name__ == "__main__":

    # generate several species
    SPECIES_LIST = []
    for idx in range(10):
        
        # randomly select a center coordinate and decay rate
        xcoord = np.random.uniform(0, 100)
        ycoord = np.random.uniform(0, 100)
        decay = np.random.uniform(0, 1)

        # init Species with random values
        spp = Species(
            origin=(xcoord, ycoord), 
            decay=decay, 
            bioclim={
                "mean-temp": (5, 5),
                "min-temp": (1, 5),
                "mean-precip": (0.1, 0.01),
            }
        )
        SPECIES_LIST.append(spp)

    # generate random environment
    ENV = Environment(
        shape=(100, 100), 
        bioclims={},
        elevation={},
    )

    # build community from species and environment
    comm = Community(ENV, SPECIES_LIST)
    print(comm)

<Community nspecies=10; nenvs=1>
